In [1]:
import re 
import os
from os.path import join

def remove_pg_boilerplate(complete_file):
    
    lines = re.split( r'\n' , complete_file )
    read_mode = 0 
    full_text = ''
    
    for line in lines:
        #print(line)
        if read_mode == 1:
            full_text += line + '\n'
            
        if re.search( r'\*{3,}\s+START\s+OF\s+TH(E|IS)\s+PROJECT\s+GUTENBERG\s+EBOOK' ,  str(line) , re.IGNORECASE ):
            read_mode = 1
        if re.search( r'\*{3,}\s+END\s+OF\s+TH(E|IS)\s+PROJECT\s+GUTENBERG\s+EBOOK' ,  str(line) , re.IGNORECASE ):
            read_mode = 0
            
    full_text = full_text.strip()
    if re.search( r'^Produced by' , full_text , re.IGNORECASE ):
        full_text = full_text[ full_text.index('\n') : len(full_text) ]
    return full_text

In [2]:
authors = ['Burney Fanny',
          'Lennox Charlotte',
           'More Hannah' , 
           'Smith Charlotte' ,
         'West Jane' ,
             'Shelley Mary Wollstonecraft' ,
     'Brontë Emily' ,
 'Herr Charlotte B. (Charlotte Bronte)' ,
           'Brontë Anne' ,
           'Richardson Samuel' ,
           'Fielding Henry' ,
             'Thackeray William Makepeace' ,
           'Trollope Anthony' ]



In [3]:
import requests
import pandas as pd

response = requests.get('https://edu.nl/xcm3q')
if response.status_code == 200:
    csv = response.text
    with open('gutenberg_metadata.csv' , 'w' , encoding = 'utf-8') as f:
        f.write(csv)

df = pd.read_csv('gutenberg_metadata.csv' )



In [4]:
corpus = 'Contemporary_authors'

if not(os.path.exists(corpus)):
    os.makedirs(corpus)

In [5]:
gutenberg_files = []

for author in authors:
    print(author)
    df_author = df[ (df['author'] == author) & (df['language'] =='en' ) ]
    print(df_author.shape)
    for i, row in df_author.iterrows():
        gutenberg_files.append( row['url'] )

Burney Fanny
(16, 7)
Lennox Charlotte
(1, 7)
More Hannah
(8, 7)
Smith Charlotte
(1, 7)
West Jane
(0, 7)
Shelley Mary Wollstonecraft
(10, 7)
Brontë Emily
(1, 7)
Herr Charlotte B. (Charlotte Bronte)
(3, 7)
Brontë Anne
(2, 7)
Richardson Samuel
(14, 7)
Fielding Henry
(15, 7)
Thackeray William Makepeace
(61, 7)
Trollope Anthony
(85, 7)


In [6]:
print(len(gutenberg_files))

217


In [7]:
for url in gutenberg_files:
    print("Downloading " + url + " ...")
    file_name = f'{os.path.basename(url)}'
    response = requests.get(url)

    if response:
        response.encoding = 'utf-8'
        full_text = remove_pg_boilerplate(response.text)  
        full_text = re.sub(r'\s+' , ' ' , full_text)
        path = join( corpus , file_name )
        out = open( path , 'w' , encoding = 'utf-8')
        out.write( full_text.strip() )
        out.close()
        

In [8]:
print('Done!')

Done!
